<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Описание проекта

Интернет-магазин запускает новый сервис, в котором пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Необходимо обучить модель классифицировать комментарии на позитивные и негативные. В нашем распоряжении набор данных с разметкой о токсичности правок.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
import pandas as pd
from pymystem3 import Mystem
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import re
from tqdm.notebook import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
import numpy as np
from sklearn.utils import shuffle

In [2]:
data = pd.read_csv('/datasets/toxic_comments.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [4]:
data.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


В данных присутствуют 159571 объект и 2 столбца. Столбец `text` содержит текст комментариев пользвателей, а столбец `toxic` информацию о токсичности комментария. Пропусков не обнаружено.

Отчистим текст от ненужных символов.

In [6]:
m = Mystem()
l = WordNetLemmatizer()

In [7]:
def clear_text(text):
    return " ".join(re.sub(r'[^a-zA-Z]', ' ', text).split())

data['text'] = data['text'].apply(clear_text)
data

,text,toxic
0,Explanation Why the edits made under my userna...,0
1,D aww He matches this background colour I m se...,0
2,Hey man I m really not trying to edit war It s...,0
3,More I can t make any real suggestions on impr...,0
4,You sir are my hero Any chance you remember wh...,0
...,...,...
159566,And for the second time of asking when your vi...,0
159567,You should be ashamed of yourself That is a ho...,0
159568,Spitzer Umm theres no actual article for prost...,0
159569,And it looks like it was actually you who put ...,0


Лемматизируем текст.

In [8]:
def get_tag(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tags_dict = {
        'J': wordnet.ADJ,
        'V': wordnet.VERB,
        'N': wordnet.NOUN,
        'R': wordnet.ADV
    }
    return tags_dict.get(tag, wordnet.NOUN)

In [9]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [10]:
def lem(val):
    return l.lemmatize(val)

tqdm.pandas()
data['text'] = data['text'].progress_apply(lambda x: " ".join(l.lemmatize(w, get_tag(w)) for w in nltk.word_tokenize(x)))
data

/opt/conda/lib/python3.7/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


,text,toxic
0,Explanation Why the edits make under my userna...,0
1,D aww He match this background colour I m seem...,0
2,Hey man I m really not try to edit war It s ju...,0
3,More I can t make any real suggestion on impro...,0
4,You sir be my hero Any chance you remember wha...,0
...,...,...
159566,And for the second time of ask when your view ...,0
159567,You should be ashamed of yourself That be a ho...,0
159568,Spitzer Umm there no actual article for prosti...,0
159569,And it look like it be actually you who put on...,0


Разделим данные на 3 выборки: тренировочную, тестовую и валидационную.

In [11]:
df = data.copy()
rnd = 123

In [12]:
train, test = train_test_split(df, test_size=0.2, random_state=rnd)
train, valid = train_test_split(train, test_size=0.25, random_state=rnd)

Создадим наборы признаков для этих выборок, учитывая стоп слова.

In [13]:
stopwords = set(nltk_stopwords.words('english'))
count_tf_idf = TfidfVectorizer(stop_words=stopwords, lowercase=True, max_features=5000)
tf_idf = count_tf_idf.fit_transform(train['text'])
tf_idf.shape

(95742, 5000)

In [14]:
tf_idf_valid = count_tf_idf.transform(valid['text'])
tf_idf_valid.shape

(31914, 5000)

In [15]:
tf_idf_test = count_tf_idf.transform(test['text'])
tf_idf_test.shape

(31915, 5000)

## Обучение

Выделим признаки и целевой признак в отдельные переменные.

In [16]:
target_train = train['toxic']
features_train = tf_idf
target_valid = valid['toxic']
features_valid = tf_idf_valid

In [17]:
features_valid.shape

(31914, 5000)

Обучим логистическую регрессию и узнаем значение метрики F1 для валидационной выборки.

In [18]:
log = LogisticRegression(random_state=rnd)
log.fit(features_train, target_train)
predictions = log.predict(features_valid)
print('F1: ', f1_score(target_valid, predictions))

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


F1:  0.7387033398821219


То же самое проделаем для модели LinearSVC.

In [19]:
f1_best = 0
iter_best = 0
for n_iter in range(1000, 5500, 500):
    lin = LinearSVC(random_state=rnd, C=0.95, max_iter=n_iter)
    lin.fit(features_train, target_train)
    predictions = lin.predict(features_valid)
    f1 = f1_score(target_valid, predictions)
    if f1_best < f1:
        f1_best = f1
        iter_best = n_iter
print('Число итераций:', iter_best)
print('F1:', f1_best)

Число итераций: 1000
F1: 0.766286098052384


In [20]:
f1_best = 0
c_best = 0
for c in np.arange(0.5, 1, 0.5):
    lin = LinearSVC(random_state=rnd, C=c, max_iter=iter_best)
    lin.fit(features_train, target_train)
    predictions = lin.predict(features_valid)
    f1 = f1_score(target_valid, predictions)
    if f1_best < f1:
        f1_best = f1
        c_best = c
print('C:', c_best)
print('F1:', f1_best)

C: 0.5
F1: 0.7629919918214347


Метрика качества удовлетворяет условию. Проверим метрику этой модели на тестовой выборке.

In [21]:
target_test = test['toxic']
features_test = tf_idf_test

predictions = lin.predict(features_test)
print('F1 на тесте:', f1_score(target_test, predictions))

F1 на тесте: 0.7722342733188721


Модель прошла тест.

## Выводы

    - Была произведена обработка данных от лишних символов
    - Проведена лемматизация с учетов стоп-слов
    - Созданы признаки на основе тренировочной выборки по TF-IDF
    - Обучены разные модели
    - На вылидационной выбоке выбрана модель LinearSVC
    - Проведено успешное тестирование этой модели.

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [x]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны